## Day 30 Lecture 1 Assignment

In this assignment, we will build our first random forest model. We will use a dataset containing churn information for a telephone company's customers.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, plot_confusion_matrix

from sklearn.ensemble import RandomForestClassifier

In [1]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 1.8 MB/s eta 0:00:011


Read in the telecom churn dataset.

https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/telecom_churn.csv

In [3]:
# answer here
url = 'https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/telecom_churn.csv'
df = pd.read_csv(url)
df.head()

,CustomerID,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,...,ReferralsMadeBySubscriber,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus
0,3000002,Yes,24.00,219.0,22.0,0.25,0.0,0.0,-157.0,-19.0,...,0,4,No,0,30,Yes,1-Highest,Suburban,Professional,No
1,3000010,Yes,16.99,10.0,17.0,0.00,0.0,0.0,-4.0,0.0,...,0,5,No,0,30,No,4-Medium,Suburban,Professional,Yes
2,3000014,No,38.00,8.0,38.0,0.00,0.0,0.0,-2.0,0.0,...,0,6,No,0,Unknown,No,3-Good,Town,Crafts,Yes
3,3000022,No,82.28,1312.0,75.0,1.24,0.0,0.0,157.0,8.1,...,0,6,No,0,10,No,4-Medium,Other,Other,No
4,3000026,Yes,17.14,0.0,17.0,0.00,0.0,0.0,0.0,-0.2,...,0,9,No,1,10,No,1-Highest,Other,Professional,Yes


Perform the following preprocessing steps:

- This dataset has many columns; identify a subset of 10-15 features that you think could affect a customer's propensity to churn (also called "customer loss" or "customer attrition"). Include at least 1 categorical variable.
- Drop all rows that containing any missing values (this should be a fairly small number of rows)
- Convert the response from a string to a binary 1-0 encoding.

In [6]:
# answer goes here
#selected columns
df_new = df[['MadeCallToRetentionTeam', 'ReferralsMadeBySubscriber', 'RetentionCalls', 'HasCreditCard', 'OwnsComputer', 'RespondsToMailOffers', 'ActiveSubs', 'MonthsInService', 'CustomerCareCalls', 'TotalRecurringCharge', 'MonthlyMinutes', 'MonthlyRevenue']]
df_new.head()

,MadeCallToRetentionTeam,ReferralsMadeBySubscriber,RetentionCalls,HasCreditCard,OwnsComputer,RespondsToMailOffers,ActiveSubs,MonthsInService,CustomerCareCalls,TotalRecurringCharge,MonthlyMinutes,MonthlyRevenue
0,Yes,0,1,Yes,Yes,Yes,1,61,0.0,22.0,219.0,24.00
1,No,0,0,Yes,Yes,Yes,1,58,0.0,17.0,10.0,16.99
2,No,0,0,Yes,No,No,1,60,0.0,38.0,8.0,38.00
3,No,0,0,Yes,No,Yes,2,59,4.3,75.0,1312.0,82.28
4,No,0,0,Yes,Yes,Yes,2,53,0.0,17.0,0.0,17.14


In [7]:
#drop rows containing nan
df_new.dropna(inplace=True)

/Users/kristofcsaba/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
#Convert target from str to binary
df['Churn'] = df['Churn'].replace({'Yes': 1, 'No':0})

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [11]:
df_new.describe(include='O')

,MadeCallToRetentionTeam,HasCreditCard,OwnsComputer,RespondsToMailOffers
count,50891,50891,50891,50891
unique,2,2,2,2
top,No,Yes,No,No
freq,49158,34397,41458,31718


Although some implementations of tree-based models support categorical variables directly, scikit-learn's implementation does not, so we will need to convert our categorical variable(s) into numbers. In linear regression, we used one-hot/dummy encoding, but this has drawbacks that can be particularly problematic in the context of trees. Namely, dummy encoding can lead to very sparse columns when dealing with high-dimensional categorical features; the tree treats these levels as indepedent features is very unlikely to make a split on any of the individual levels, since the corresponding columns consist almost entirely consist of a single value.

Ideally, we would consider all of the levels simultaneously, which trees can do efficiently. However, sklearn does not implement this efficient algorithm for splitting on categorical variables. In this situation, a better alternative to dummy encoding is leave-one-out-encoding. This transforms each categorical into its conditional probability of the positive class, excluding the observation itself to avoid potential data leakage. Perform the following steps:

- Create an 80-20 train/test split on the data
- Fit a leave-one-out encoder to the training data and encode the categorical variables in the training data. The category_encoders package is helpful for this task. (Note that this ordering is important; if we fitted the encoder to the full dataset before making the train/test split, we would introduce data leakage by allowing the test data to affecting the estimates of the category-conditional probabilities.)

In [ ]:
# answer goes here





Next, fit the random forest model. You can tinker with or tune the parameters if you like, although we will use the default parameter values.

In [ ]:
# answer goes here





To evaluate our model, produce the predicted values for the test sample. Remember to apply our previously fitted encoder to the test data prior to making predictions.

In [ ]:
# answer goes here





After making the predictions, print out the classification report (sklearn.metrics.classification_report). In addition, plot the confusion matrix and ROC curve. How did our model perform?

In [ ]:
# answer goes here





Print the feature importances, ordered by importance from high to low. What is the default sklearn feature importance metric?

In [ ]:
# answer goes here



